In [2]:
import pymysql.cursors
import pymysql
import pandas as pd
from unidecode import unidecode

# Class

In [3]:
class Outsider:
    candidaturas = pd.DataFrame()
    deputados = pd.DataFrame()
    ano = ""
    
    def __init__(self, ano):
        self.__setAno(ano)
        self.__setDeputados()
        
    def __setAno(self, ano):
        self.ano = ano
        
    
    def getDeputados(self):
        return self.deputados;
        
    def __selectDeputados(self):
        query = '''
            SELECT
                NOME_CANDIDATO,
                DESCRICAO_CARGO,
                SIGLA_UF,
                NUMERO_CPF_CANDIDATO,
                NUMERO_TITULO_ELEITORAL_CANDIDATO,
                ANO_ELEICAO,
                SIGLA_PARTIDO,
                DESCRICAO_SITUACAO_TOTAL_TURNO
            FROM CANDIDATURAS
            WHERE
                ANO_ELEICAO = {} AND
                DESCRICAO_CARGO = 'DEPUTADO FEDERAL' AND
                (DESCRICAO_SITUACAO_TOTAL_TURNO LIKE 'ELEITO%' OR
                 DESCRICAO_SITUACAO_TOTAL_TURNO LIKE 'MEDIA');
            '''
        deputados = self.query2pandas(query.format(self.ano))
        deputados = deputados.drop_duplicates()
        
        return(deputados)
    
    
    def __evaluateOutsiders(self, deputados):
        deputados['Outsider_Type'] = 0
        # 0 não Outisder
        # 1 nunca se candidatou nem venceu
        # 2 ja se candidatou mas não venceu
        # 3 experiecia estadual
        # 4 experiencia municipal
        for candidato_cpf in list(set(deputados['NUMERO_CPF_CANDIDATO'])):
            # nunca se candidatou nem venceu
            if not (self.__beenCandidate(candidato_cpf, 'DEPUTADO FEDERAL') or
                  self.__beenCandidate(candidato_cpf, 'DEPUTADO ESTADUAL') or
                  self.__beenCandidate(candidato_cpf, 'GOVERNADOR') or
                  self.__beenCandidate(candidato_cpf, 'PREFEITO') or
                  self.__beenCandidate(candidato_cpf, 'VEREADOR') or
                  self.__beenElected(candidato_cpf, 'DEPUTADO FEDERAL') or
                  self.__beenElected(candidato_cpf, 'DEPUTADO ESTADUAL') or
                  self.__beenElected(candidato_cpf, 'GOVERNADOR') or
                  self.__beenElected(candidato_cpf, 'PREFEITO') or
                  self.__beenElected(candidato_cpf, 'VEREADOR')):
                deputados.loc[deputados['NUMERO_CPF_CANDIDATO'] == candidato_cpf, 'Outsider_Type'] = 'Outsider_1'

                ## se candidatou a alguma coisa e perdeu
            elif (self.__beenCandidate(candidato_cpf, 'DEPUTADO FEDERAL') or
                  self.__beenCandidate(candidato_cpf, 'DEPUTADO ESTADUAL') or
                  self.__beenCandidate(candidato_cpf, 'GOVERNADOR') or
                  self.__beenCandidate(candidato_cpf, 'PREFEITO') or
                  self.__beenCandidate(candidato_cpf, 'VEREADOR')) and \
                  not (self.__beenElected(candidato_cpf, 'DEPUTADO FEDERAL') or
                  self.__beenElected(candidato_cpf, 'DEPUTADO ESTADUAL') or
                  self.__beenElected(candidato_cpf, 'GOVERNADOR') or
                  self.__beenElected(candidato_cpf, 'PREFEITO') or
                  self.__beenElected(candidato_cpf, 'VEREADOR')):
                deputados.loc[deputados['NUMERO_CPF_CANDIDATO'] == candidato_cpf, 'Outsider_Type'] = 'Outsider_2'

            elif not self.__beenElected(candidato_cpf, 'DEPUTADO FEDERAL') and \
                  (self.__beenElected(candidato_cpf, 'DEPUTADO ESTADUAL') or \
                  self.__beenElected(candidato_cpf, 'GOVERNADOR')) and \
               not (self.__beenElected(candidato_cpf, 'PREFEITO') or
                  self.__beenElected(candidato_cpf, 'VEREADOR')):
                deputados.loc[deputados['NUMERO_CPF_CANDIDATO'] == candidato_cpf, 'Outsider_Type'] = 'Outsider_3'

            elif not self.__beenElected(candidato_cpf, 'DEPUTADO FEDERAL') and \
                  (self.__beenElected(candidato_cpf, 'PREFEITO') or
                  self.__beenElected(candidato_cpf, 'VEREADOR')):
                deputados.loc[deputados['NUMERO_CPF_CANDIDATO'] == candidato_cpf, 'Outsider_Type'] = 'Outsider_4'

            else:
                deputados.loc[deputados['NUMERO_CPF_CANDIDATO'] == candidato_cpf, 'Outsider_Type'] = 'Outsider_0'

        return deputados
    
    
    def __setCandidaturas(self):
        query = '''
            SELECT
                NOME_CANDIDATO,
                DESCRICAO_CARGO,
                SIGLA_UF,
                NUMERO_CPF_CANDIDATO,
                NUMERO_TITULO_ELEITORAL_CANDIDATO,
                ANO_ELEICAO,
                SIGLA_PARTIDO,
                DESCRICAO_SITUACAO_TOTAL_TURNO
            FROM CANDIDATURAS
            WHERE
                ANO_ELEICAO < {} AND
                {};
        '''.format(self.ano, ' OR\n'.join(['NUMERO_CPF_CANDIDATO = "{}"'.format(e) for e in self.deputados.NUMERO_CPF_CANDIDATO]))

        candidaturas = self.query2pandas(query)
        candidaturas.loc[candidaturas['DESCRICAO_SITUACAO_TOTAL_TURNO'] == 'ELEITO POR MEDIA', 'DESCRICAO_SITUACAO_TOTAL_TURNO'] = 'ELEITO'
        candidaturas.loc[candidaturas['DESCRICAO_SITUACAO_TOTAL_TURNO'] == 'ELEITO POR QP', 'DESCRICAO_SITUACAO_TOTAL_TURNO'] = 'ELEITO'
        self.candidaturas = candidaturas
        
        
    def __evaluatePartido(self):
        partyTypology = pd.read_csv('../assets/partyTypology.csv')
        self.deputados = self.deputados.join(partyTypology.set_index(['SIGLA_PARTIDO', 'ANO']), on=['SIGLA_PARTIDO', 'ANO_ELEICAO'])
    
    
    def __setDeputados(self):
        self.deputados = self.__selectDeputados()
        
        self.__setCandidaturas()
        
        self.deputados = self.__evaluateOutsiders(self.deputados)
        self.__evaluatePartido()
        
        self.candidaturas = pd.DataFrame()
    
    def query2pandas(self, query):
        # MySQL Connection
        connection = pymysql.connect(host='localhost',
                                user='root',
                                password='banana123',
                                db='OUTSIDERS',
                                charset='utf8')
        try:
            data = pd.read_sql(query, connection)
        except:
            print('ERROR')
        finally:
            return data
        connection.close()
        
        
    def __beenElected(self, candidato_cpf, cargo):
        past_elections = len(self.candidaturas[(self.candidaturas['NUMERO_CPF_CANDIDATO'] == candidato_cpf) &
                         (self.candidaturas['ANO_ELEICAO'] < self.ano) &
                         (self.candidaturas['DESCRICAO_CARGO'] == cargo) &
                         (self.candidaturas['DESCRICAO_SITUACAO_TOTAL_TURNO'] == 'ELEITO')])
        return past_elections > 0


    def __beenCandidate(self, candidato_cpf, cargo):
        past_elections = len(self.candidaturas[(self.candidaturas['NUMERO_CPF_CANDIDATO'] == candidato_cpf) &
                         (self.candidaturas['ANO_ELEICAO'] < self.ano) &
                         (self.candidaturas['DESCRICAO_CARGO'] == cargo) &
                         (self.candidaturas['DESCRICAO_SITUACAO_TOTAL_TURNO'] != 'ELEITO')])
        return past_elections > 0

In [4]:
outsider.getDeputados()

NameError: name 'outsider' is not defined

In [5]:
outsiders_df = pd.DataFrame()

for year in range(1994, 2020, 4):
    
    # geral
    print(year, '...')
    outsider = Outsider(year)
    outsiders_df = outsiders_df.append(outsider.getDeputados())
    

1994 ...
1998 ...
2002 ...
2006 ...
2010 ...
2014 ...
2018 ...


In [6]:
%%time

counts_geral = pd.DataFrame()
counts_partido = pd.DataFrame()
counts_tipoPartido = pd.DataFrame()  
for year in range(1994, 2020, 4):
    count = outsiders_df.loc[outsiders_df['ano'] == ano, 'Outsider_Type'].value_counts()
    count = pd.DataFrame(
            data={
                'value':count.index,
                year:count.values
            })
    count = count.sort_values('value', ascending=True)
    count = count.set_index('value')
    
    counts_geral = counts_geral.append(count.transpose())
    print(year, 'Count Geral Ok\n')
    
    
    # por partido
    count = outsider.getDeputados().groupby('SIGLA_PARTIDO')['Outsider_Type'].value_counts().unstack()
    count['partido'] = ['{}'.format(e) for e in count.index]
    count['ano'] = year
    count['partido_ano'] = ['{}_{}'.format(e, year) for e in count.index]
    count.fillna(0, inplace=True)
    count.index = range(0, len(count))
    
    counts_partido = counts_partido.append(count)
    print(year, 'Count Partido Ok\n')
    
    # por tipo de partido
    count = outsider.getDeputados().groupby('TIPO')['Outsider_Type'].value_counts().unstack()
    count['tipoPartido'] = ['{}'.format(e) for e in count.index]
    count['ano'] = year
    count['tipoPartido_ano'] = ['{}_{}'.format(e, year) for e in count.index] 
    count.fillna(0, inplace=True)
    count.index = range(0, len(count))
    
    counts_tipoPartido = counts_tipoPartido.append(count)
    print(year, 'Count Tipo Ok\n')
    
    #print(counts_geral)
    #print(counts_partido)
    #print(counts_tipoPartido)
    

NameError: name 'outsiders' is not defined

In [7]:
## some cleaning

In [8]:
counts_geral.fillna(0, inplace=True)
counts_partido.fillna(0, inplace=True)
counts_tipoPartido.fillna(0, inplace=True)

counts_partido = counts_partido[['partido_ano', 'ano', 'partido', 'Outsider_0', 'Outsider_1', 'Outsider_2', 'Outsider_3', 'Outsider_4']]
counts_tipoPartido = counts_tipoPartido[['tipoPartido_ano', 'ano', 'Outsider_0', 'Outsider_1', 'Outsider_2', 'Outsider_3', 'Outsider_4']]
    

KeyError: "None of [Index(['partido_ano', 'ano', 'partido', 'Outsider_0', 'Outsider_1',\n       'Outsider_2', 'Outsider_3', 'Outsider_4'],\n      dtype='object')] are in the [columns]"

In [ ]:
counts_geral.to_excel('../data/output/frequencia de outsiders por ano 1994-2018.xlsx', index=False)
counts_partido.to_excel('../data/output/frequencia de outsiders por partido por ano 1994-2018.xlsx', index=False)
counts_tipoPartido.to_excel('../data/output/frequencia de outsiders por tipo de partido por ano 1994-2018.xlsx', index=False)

counts_geral.to_csv('../data/output/frequencia de outsiders por ano 1994-2018.csv')
counts_partido.to_csv('../data/output/frequencia de outsiders por partido por ano 1994-2018.csv', index=False)
counts_tipoPartido.to_csv('../data/output/frequencia de outsiders por tipo de partido por ano 1994-2018.csv', index=False)

# Correção com a Planilha do Adriano - deputados_eleutos

In [ ]:
outsiders_df

In [ ]:
deputados_eleutos = pd.read_excel('../assets/19.10.2019 -  EXCEL BD só DEP-FED_eleitos_1998-2018 corr.xlsx')
deputados_eleutos['NOME_CANDIDATO'] = [e.upper() for e in deputados_eleutos['Nome_candidato']]
deputados_eleutos

In [ ]:
novo = deputados_eleutos.join(outsiders_df[['ANO_ELEICAO', 'NOME_CANDIDATO', 'Outsider_Type', 'TIPO']].set_index(['ANO_ELEICAO', 'NOME_CANDIDATO']), on=['ano_eleicao', 'NOME_CANDIDATO'], how='left')

In [ ]:
novo = novo.loc[:,[e != 'NOME_CANDIDATO' for e in novo.columns.values]]
novo = novo.drop_duplicates()
novo

In [ ]:
novo.to_excel('../data/27.11.2019 -  EXCEL BD só DEP-FED_eleitos_1998-2018 corr - TIPO OUTSIDER E TIPO PARTIDO.xlsx', index=False)

In [ ]:
deputadosEleitosSQL[(deputadosEleitosSQL['ANO_ELEICAO'] >=1998) & (deputadosEleitosSQL['ANO_ELEICAO'] <= 2018)]
deputadosEleitosSQL

In [ ]:
deputadosEleitosSQL.groupby('ANO_ELEICAO')['NOME_CANDIDATO'].size()

In [ ]:
outsider

In [ ]:
outsider = Outsider(2002)
outsider = outsider.getDeputados()

In [ ]:
for ano in range(1998,2020,2):
    nomes_1 = outsider.loc[outsider['ANO_ELEICAO'] == ano, 'NOME_CANDIDATO']
    nomes_1
    nomes_2 = deputados_eleutos.loc[deputados_eleutos['ano_eleicao'] == ano, 'Nome_candidato']
    nomes_2 = [unidecode(e.upper()) for e in nomes_2]
    nomes_2
    
    print(ano)
    print(len(nomes_1))
    print(len(nomes_2))

    print('diferentes 1')
    diferentes = list(set(nomes_1).difference(nomes_2))
    print(diferentes)
    
    print('diferentes 2')
    diferentes = list(set(nomes_2).difference(nomes_1))
    print(diferentes)
    
    for diferente in diferentes:
        outsider.loc[(outsider['NOME_CANDIDATO'] == diferente) & (outsider['ANO_ELEICAO'] == ano), 'DESCRICAO_SITUACAO_TOTAL_TURNO'] = 'NÃO ELEITO'
    print('_____')

In [ ]:
## tira 94 e 98
counts_geral.loc[2018,:]

In [ ]:
counts_partido

In [10]:
counts_tipoPartido

,tipoPartido_ano,ano,Outsider_0,Outsider_1,Outsider_2,Outsider_3,Outsider_4
0,CENTRO/ESQUERDA_1994,1994,0.0,141.0,0.0,0.0,0.0
1,NOVA DIREITA_1994,1994,0.0,14.0,0.0,0.0,0.0
2,PERSONALISTA_1994,1994,0.0,2.0,0.0,0.0,0.0
3,VELHA DIREITA_1994,1994,0.0,70.0,0.0,0.0,0.0
0,CENTRO/ESQUERDA_1998,1998,3.0,296.0,0.0,0.0,0.0
1,NOVA DIREITA_1998,1998,1.0,31.0,0.0,0.0,0.0
2,VELHA DIREITA_1998,1998,2.0,184.0,0.0,0.0,0.0
0,CENTRO/ESQUERDA_2002,2002,130.0,53.0,74.0,33.0,24.0
1,NOVA DIREITA_2002,2002,14.0,4.0,6.0,2.0,0.0
2,VELHA DIREITA_2002,2002,90.0,34.0,28.0,17.0,8.0


In [11]:
count['0'].sum()

KeyError: '0'

In [47]:
outsiders_df.loc[['ALEXANDRE BAL' in e for e in outsiders_df['NOME_CANDIDATO']],:]

,NOME_CANDIDATO,DESCRICAO_CARGO,SIGLA_UF,NUMERO_CPF_CANDIDATO,NUMERO_TITULO_ELEITORAL_CANDIDATO,ANO_ELEICAO,SIGLA_PARTIDO,DESCRICAO_SITUACAO_TOTAL_TURNO,Outsider_Type,TIPO
446,ALEXANDRE BALDY DE SANT ANNA BRAGA,DEPUTADO FEDERAL,GO,87342235172,038336711058,2014,PSDB,ELEITO POR QP,Outsider_1,CENTRO/ESQUERDA
